# Notice about this Fama Macbeth cross sectional regression
This method here uses total common factor as the risk premia and not the factor portfolio against the test portfolios instead of the assets

In [1]:
%matplotlib inline
import pandas as pd
import seaborn as sns
from tsmom_model import *
import PPCA_MR3 as p
import numpy as np
from datetime import datetime
from linearmodels.asset_pricing import LinearFactorModel

In [2]:
import pylab
sns.set(font_scale=3)
pylab.rcParams['figure.figsize'] = (24, 18)

In [3]:
cleansed=load_amihud_markets_price()

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:158: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data=pd.read_csv('./Price.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:160: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  qd=pd.read_csv('./liquid_contracts.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='last')


In [4]:
amihud=calculate_amihud_liquidity(cleansed)[:'2016'] 

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:42: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mean()
  data=pd.read_csv('./Volume.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='mean')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:44: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  v=pd.read_csv('./volume_data.csv',index_col=0,parse_dates=['Date']).resample(rule='d',how='sum')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:46: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=250,min_periods=100,center=False).mean()
  volume=pd.rolling_mean(v,250,min_periods=100).resample(rule='d',how='mean')[:'2016']
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:46: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).mea

# Amihud (2002) Annual portfolio sorts

In [5]:
data=calc_zscore_expanding_window(amihud.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:273: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).mean()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:273: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).std()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)


In [6]:
sector_portfolio_sorts={}
for s in sector_zscores.keys():
    sector_portfolio_sorts[s]=quantile_portfolios_annual(sector_zscores[s],cleansed,3)

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:238: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).median()
  mkts=quantile_columns(rank_data.resample(rule='a',how='median'),year,number_of_buckets,i)
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:239: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  rtns = price_data.resample(rule='m',how='last')[mkts].pct_change()[str(y+1)].mean(axis=1)


In [7]:
FF=pd.read_csv('FF.csv',parse_dates=['Date'],index_col=0).resample(rule='m',how='sum')/100.

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).sum()
  """Entry point for launching an IPython kernel.


In [8]:
sector_rtns={}
d_map=sector_map(amihud)
for sect in d_map.keys():
    sector_rtns[sect]=np.log(cleansed[d_map[sect]].resample(rule='m',how='last')).diff().dropna(how='all')

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  after removing the cwd from sys.path.


In [10]:
data=calc_zscore_expanding_window(amihud.dropna(how='all'),3).dropna(how='all')
no_pc=3
PPCA=p.PPCA()
PPCA.fit(np.array(data),d=no_pc)
pcs=pd.DataFrame()
for i in range(0,no_pc,1):
    x=pd.DataFrame(PPCA.C.T[i]*PPCA.data).sum(axis=1)
    x.set_axis(labels=data.index,axis='index')
    pcs[i]=x
sector_PC = pcs

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:273: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).mean()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:273: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).std()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)


# Expirimenting with annual portfolio sorts
* PPCA common liquidity factor per asset
* Median Amihud value
* Whole market Amihud value

However the strongest looks like the first one, but nothing is strongerly predictive.

However there is a timing mismatch because this is using annual portfolio sorts

In [11]:
final_res=pd.DataFrame()

In [12]:
portfolios=pd.DataFrame()
for sec in ['Equities', 'Commodities', 'Fixed Income']:
    for i in sector_portfolio_sorts[sec].columns:
        portfolios[sec + " "+ str(i)]=sector_portfolio_sorts[sec][i]

In [14]:
portfolios=pd.DataFrame()
for sec in ['Equities', 'Commodities', 'Fixed Income']:
    for i in sector_portfolio_sorts[sec].columns:
        portfolios[sec + " "+ str(i)]=sector_portfolio_sorts[sec][i]
        
ind=portfolios.index
factor=pd.DataFrame()
factor=get_aqr_factors('GLOBAL').loc[ind]
#factor=pd.DataFrame()
factor['MKT']=cleansed.resample(rule='m',how='last').pct_change().mean(axis=1).loc[ind]
factor['Liquidity']=sector_PC.sum(axis=1).loc[ind]
mod=LinearFactorModel(portfolios.dropna(),factor.loc[ind])
res=mod.fit(cov_type='kernel')
final_res['A-A beta']=res.risk_premia
final_res['A-A tstat']=res.risk_premia_tstats
print(res)

                      LinearFactorModel Estimation Summary                      
No. Test Portfolios:                  9   R-squared:                      0.5843
No. Factors:                          4   J-statistic:                    6.8138
No. Observations:                   204   P-value                         0.2349
Date:                  Tue, Apr 03 2018   Distribution:                  chi2(5)
Time:                          19:31:18                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
VAL            0.0014     0.0089     0.1564     0.8757     -0.0160      0.0187
MOM           -0.0035     0.0137    

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  # Remove the CWD from sys.path while we load stuff.


# Monthly Amihud sorts

In [15]:
mon_amihud=read_monthly(amihud=True)

In [17]:
portfolios=pd.DataFrame()
for sec in ['Equities', 'Commodities', 'Fixed Income']:
    for i in mon_amihud[sec].columns:
        portfolios[sec + " "+ str(i)]=mon_amihud[sec][i]
        
ind=portfolios.index
factor=pd.DataFrame()
factor=get_aqr_factors('GLOBAL').loc[ind]
#factor=pd.DataFrame()
factor['MKT']=cleansed.resample(rule='m',how='last').pct_change().mean(axis=1).loc[ind]
factor['Liquidity']=sector_PC.sum(axis=1).loc[ind]
#factor['Median Sector Amihud']=amihud[d_map[sec]].median(axis=1).dropna()
#factor['Median total Amihud']=amihud.median(axis=1).dropna()
#factor['Mkt-RF']=FF['Mkt-RF']
mod=LinearFactorModel(portfolios.dropna(),factor.loc[ind])
res=mod.fit(cov_type='kernel')
final_res['A-M beta']=res.risk_premia
final_res['A-M tstat']=res.risk_premia_tstats
print(res)

                      LinearFactorModel Estimation Summary                      
No. Test Portfolios:                  9   R-squared:                      0.5869
No. Factors:                          4   J-statistic:                    12.703
No. Observations:                   192   P-value                         0.0263
Date:                  Tue, Apr 03 2018   Distribution:                  chi2(5)
Time:                          19:31:31                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
VAL           -0.0016     0.0052    -0.3085     0.7577     -0.0117      0.0085
MOM            0.0016     0.0065    

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  # Remove the CWD from sys.path while we load stuff.


# FHT method

In [18]:
FHT=calculate_FHT(cleansed)
data=calc_zscore_expanding_window(FHT.dropna(how='all'),3).dropna(how='all')
sector_zscores={}
d_map=sector_map(FHT)
for sect in d_map.keys():
    sector_zscores[sect]=data[d_map[sect]].dropna(how='all')

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:283: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  nonzero=x[x==0].resample(rule='m',how='count')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:284: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).count()
  Z=(nonzero/x.resample(rule='m',how='count')).dropna()
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:285: FutureWarning: pd.rolling_std is deprecated for Series and will be removed in a future version, replace with 
	Series.rolling(window=12,center=False).std()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(rule='m',how='last')
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:285: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  data[c]=pd.Series(norm.cdf((1+Z)/2.),index=Z.index)*2*pd.rolling_std(x,12).resample(

In [19]:
sector_portfolio_sorts_FHT={}
for s in sector_zscores.keys():
    sector_portfolio_sorts_FHT[s]=quantile_portfolios_annual(sector_zscores[s],cleansed,5)

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:238: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).median()
  mkts=quantile_columns(rank_data.resample(rule='a',how='median'),year,number_of_buckets,i)
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:239: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  rtns = price_data.resample(rule='m',how='last')[mkts].pct_change()[str(y+1)].mean(axis=1)


In [20]:
data=calc_zscore_expanding_window(FHT.dropna(how='all'),3).dropna(how='all')

no_pc=3
PPCA=p.PPCA()
PPCA.fit(np.array(data),d=no_pc)
pcs=pd.DataFrame()
for i in range(0,no_pc,1):
    x=pd.DataFrame(PPCA.C.T[i]*PPCA.data).sum(axis=1)
    x.set_axis(labels=data.index,axis='index')
    pcs[i]=x
sector_PC = pcs

/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:273: FutureWarning: pd.rolling_mean is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).mean()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)
/Users/mrefermat/git/FinancePhD/LiquidityMomentum/tsmom_model.py:273: FutureWarning: pd.rolling_std is deprecated for DataFrame and will be removed in a future version, replace with 
	DataFrame.rolling(window=100000000,min_periods=3,center=False).std()
  return (df-pd.rolling_mean(df,100000000,min_periods=min_per))/pd.rolling_std(df,100000000,min_periods=min_per)


# Using Anual sorts

In [21]:
portfolios=pd.DataFrame()
for sec in ['Equities', 'Commodities', 'Fixed Income']:
    for i in mon_amihud[sec].columns:
        portfolios[sec + " "+ str(i)]=sector_portfolio_sorts_FHT[sec][i]
        
ind=portfolios.index
factor=pd.DataFrame()
factor=get_aqr_factors('GLOBAL').loc[ind]
#factor=pd.DataFrame()
factor['MKT']=cleansed.resample(rule='m',how='last').pct_change().mean(axis=1).loc[ind]
factor['Liquidity']=sector_PC.sum(axis=1).loc[ind]
mod=LinearFactorModel(portfolios.dropna(),factor.loc[ind])
res=mod.fit(cov_type='kernel')
final_res['F-A beta']=res.risk_premia
final_res['F-A tstat']=res.risk_premia_tstats
print(res)

                      LinearFactorModel Estimation Summary                      
No. Test Portfolios:                  9   R-squared:                      0.3986
No. Factors:                          4   J-statistic:                    3.5810
No. Observations:                   301   P-value                         0.6112
Date:                  Tue, Apr 03 2018   Distribution:                  chi2(5)
Time:                          19:32:21                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
VAL           -0.0055     0.0861    -0.0637     0.9492     -0.1742      0.1632
MOM            0.0035     0.0816    

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  # Remove the CWD from sys.path while we load stuff.


# Using Monthly

In [22]:
FHT_mon=read_monthly(amihud=False)

In [24]:
portfolios=pd.DataFrame()
for sec in ['Equities', 'Commodities', 'Fixed Income']:
    for i in mon_amihud[sec].columns:
        portfolios[sec + " "+ str(i)]=FHT_mon[sec][i]
        
ind=portfolios.index
factor=pd.DataFrame()
factor=get_aqr_factors('GLOBAL').loc[ind]
#factor=pd.DataFrame()
factor['MKT']=cleansed.resample(rule='m',how='last').pct_change().mean(axis=1).loc[ind]
factor['Liquidity']=sector_PC.sum(axis=1).loc[ind]
mod=LinearFactorModel(portfolios.dropna(),factor.loc[ind])
res=mod.fit(cov_type='kernel')
final_res['F-M beta']=res.risk_premia
final_res['F-M tstat']=res.risk_premia_tstats
print(res)

                      LinearFactorModel Estimation Summary                      
No. Test Portfolios:                  9   R-squared:                      0.4083
No. Factors:                          4   J-statistic:                    7.7004
No. Observations:                   300   P-value                         0.1735
Date:                  Tue, Apr 03 2018   Distribution:                  chi2(5)
Time:                          19:32:22                                         
Cov. Estimator:                  kernel                                         
                                                                                
                            Risk Premia Estimates                             
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
VAL            0.0102     0.0352     0.2892     0.7724     -0.0587      0.0791
MOM           -0.0078     0.0291    

/Users/mrefermat/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:10: FutureWarning: how in .resample() is deprecated
the new syntax is .resample(...).last()
  # Remove the CWD from sys.path while we load stuff.


In [28]:
print(final_res.round(2).T.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &   VAL &   MOM &   MKT &  Liquidity \\
\midrule
A-A beta  &  0.00 & -0.00 &  0.01 &       3.33 \\
A-A tstat &  0.16 & -0.26 &  2.29 &       0.42 \\
A-M beta  & -0.00 &  0.00 &  0.00 &       0.55 \\
A-M tstat & -0.31 &  0.25 &  1.84 &       0.31 \\
F-A beta  & -0.01 &  0.00 &  0.00 &      -0.74 \\
F-A tstat & -0.06 &  0.04 &  1.76 &      -0.04 \\
F-M beta  &  0.01 & -0.01 &  0.01 &       1.64 \\
F-M tstat &  0.29 & -0.27 &  3.08 &       0.35 \\
\bottomrule
\end{tabular}

